In [200]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [229]:
# read scrabble dictionary to wordlist
with open('scrabble_dict.txt') as f:
    wordlist = f.read().splitlines()[2:]

# filter for 5 letter words / no plurals (Wordle doesn't use them)
temp = [word for word in wordlist if len(word) == 5 ]
temp = [word for word in temp if word[4] != 'S' or word[3] == 'S']

# store alphabet as a list
alphabet = ['A', 'B','C','D','E','F',
            'G','H','I','J','K','L',
            'M','N','O','P','Q','R',
            'S','T','U','V','W','X',
            'Y','Z']

# store potential letter positions as a list
positions = ['pos_1','pos_2','pos_3','pos_4','pos_5']

In [202]:
def ltr_density(temp):
    # calculate frequency of letters in each position for remaining words
    ltr_den = np.zeros((26,5))
    
    alphabet = ['A', 'B','C','D','E','F',
                'G','H','I','J','K','L',
                'M','N','O','P','Q','R',
                'S','T','U','V','W','X',
                'Y','Z']
    positions = ['pos_1','pos_2','pos_3','pos_4','pos_5']
    
    # df to hold density counts
    df = pd.DataFrame(ltr_den, columns = positions,index=alphabet)

    #iterate through temp to count letter freq in each position
    for i in tqdm(range(len(temp))):
        for j in range(0,5):
            for k in range(len(alphabet)):
                if temp[i][j] == alphabet[k]:
                    df[positions[j]][k]+=1.0
    return df

In [203]:
def update_wd_list(contains, excludes, pos_str, non_pos, temp):
    for i in range(len(contains)):
        temp = [word for word in temp if contains[i] in word]

    for i in range(len(excludes)):
        temp = [word for word in temp if excludes[i] not in word]

    for i in range(len(pos_str)):
        if pos_str[i] != '?':
            temp = [word for word in temp if word[i] == pos_str[i]]

    for i in range(len(non_pos)):
        if len(non_pos[i]) >0:
            for j in range(len(non_pos[i])):
                if non_pos[i][j] != '?':
                    temp = [word for word in temp if word[j] != non_pos[i][j]]
    return temp


In [204]:
def guess_wd(temp, alphabet):
    top_wd = ''
    top_score = 0
    for i in tqdm(range(len(temp))):
        wd = temp[i]
        wd_score = 0
        for j in range(len(wd)):
            for k in range(len(alphabet)):
                if wd[j] == alphabet[k]:
                    wd_score = wd_score + df[positions[j]][k]
        if wd_score > top_score:
            s = set(wd)
            if len(s) == 5:
                top_wd = wd
                top_score = wd_score
    return top_wd

In [222]:
#### Interface

# ID known letters
input_contains = input("Enter letters the word contains separated by a space: ")
contains = input_contains.split()

# ID known excluded letters
input_excludes = input("Enter excluded letters separated by a space: ")
excludes = input_excludes.split()

# ID known positions
pos_str = input("Enter known positions (e.g., S??A?): ")

# ID known non-positions
non_pos =["","","","",""]
for i in range(len(contains)):
    if contains[i] not in pos_str:
        non_pos[i] = input(f"Enter known nonpositions for letter {contains[i]} (e.g., R??R?): ")

# update frequency counts, remaining word list and generate new guess
df = ltr_density(temp)
temp = update_wd_list(contains, excludes, pos_str, non_pos, temp)
guess = guess_wd(temp, alphabet)
if len(guess)<5:
    guess = temp[0]
print(f"You should try: {guess}")

Enter letters the word contains separated by a space: T R U S
Enter excluded letters separated by a space: A M E Y P O I N G
Enter known positions (e.g., S??A?): TRU?S


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1002.22it/s]

You should try: TRUSS


In [218]:
temp

['TRUSS']